# Lookout Device Compliance Threat Hunting

This notebook provides threat hunting queries for investigating device compliance and security posture issues.

## Prerequisites
- Lookout data connector configured in Microsoft Sentinel
- LookoutEvents parser deployed
- DEVICE events ingested

## 1. Non-Compliant Devices Overview

Identify devices that are not meeting compliance requirements.

In [1]:
LookoutEvents
| where EventType == "DEVICE"
| where DeviceComplianceStatus in ("Non-Compliant", "Pending")
| summarize arg_max(TimeGenerated, *) by DeviceGuid
| project 
    DeviceEmailAddress,
    DevicePlatform,
    DeviceManufacturer,
    DeviceModel,
    DeviceOSVersion,
    DeviceActivationStatus,
    DeviceComplianceStatus,
    LastSeen = TimeGenerated
| sort by LastSeen desc

DeviceEmailAddress,DevicePlatform,DeviceManufacturer,DeviceModel,DeviceOSVersion,DeviceActivationStatus,DeviceComplianceStatus,LastSeen
john.smith@contoso.com,ANDROID,Samsung,Galaxy S21,11.0,ACTIVE,Non-Compliant,2025-01-05T14:30:00Z
sarah.jones@contoso.com,ANDROID,Google,Pixel 5,10.0,ACTIVE,Non-Compliant,2025-01-05T12:15:00Z
mike.wilson@contoso.com,IOS,Apple,iPhone 11,14.8,ACTIVE,Pending,2025-01-05T10:45:00Z
lisa.chen@contoso.com,ANDROID,OnePlus,OnePlus 9,11.0,ACTIVE,Non-Compliant,2025-01-04T16:20:00Z
david.brown@contoso.com,ANDROID,Xiaomi,Mi 11,10.0,ACTIVE,Non-Compliant,2025-01-04T09:00:00Z
emma.davis@contoso.com,IOS,Apple,iPhone XR,14.4,ACTIVE,Pending,2025-01-03T15:30:00Z


## 2. Outdated Operating Systems

Find devices running potentially vulnerable OS versions.

In [2]:
LookoutEvents
| where EventType == "DEVICE"
| summarize arg_max(TimeGenerated, *) by DeviceGuid
| extend OSMajorVersion = toint(split(DeviceOSVersion, ".")[0])
| extend IsOutdated = case(
    DevicePlatform == "ANDROID" and OSMajorVersion < 12, true,
    DevicePlatform == "IOS" and OSMajorVersion < 15, true,
    false
)
| where IsOutdated == true
| summarize DeviceCount = count() by DevicePlatform, DeviceOSVersion
| sort by DeviceCount desc

DevicePlatform,DeviceOSVersion,DeviceCount
ANDROID,10.0,45
ANDROID,11.0,38
IOS,14.8,28
IOS,14.4,22
ANDROID,9.0,15
IOS,13.7,12
ANDROID,8.1,8


## 3. Device Security Status Distribution

Understand the security posture across your mobile fleet.

In [3]:
LookoutEvents
| where EventType == "DEVICE"
| summarize arg_max(TimeGenerated, *) by DeviceGuid
| summarize Count = count() by DeviceSecurityStatus, DevicePlatform
| sort by Count desc

DeviceSecurityStatus,DevicePlatform,Count
SECURE,ANDROID,245
SECURE,IOS,198
AT_RISK,ANDROID,42
AT_RISK,IOS,18
COMPROMISED,ANDROID,12
COMPROMISED,IOS,3


## 4. Inactive Devices

Identify devices that haven't checked in recently.

In [4]:
LookoutEvents
| where EventType == "DEVICE"
| where DeviceActivationStatus == "INACTIVE"
| summarize arg_max(TimeGenerated, *) by DeviceGuid
| project 
    DeviceEmailAddress,
    DevicePlatform,
    DeviceManufacturer,
    DeviceModel,
    DeviceActivationStatus,
    LastSeen = TimeGenerated
| sort by LastSeen asc

DeviceEmailAddress,DevicePlatform,DeviceManufacturer,DeviceModel,DeviceActivationStatus,LastSeen
former.employee@contoso.com,ANDROID,Samsung,Galaxy S20,INACTIVE,2024-11-15T08:30:00Z
old.device@contoso.com,IOS,Apple,iPhone 8,INACTIVE,2024-11-20T14:45:00Z
lost.phone@contoso.com,ANDROID,Google,Pixel 4,INACTIVE,2024-12-01T09:15:00Z
retired.user@contoso.com,ANDROID,OnePlus,OnePlus 7T,INACTIVE,2024-12-10T11:00:00Z
temp.contractor@contoso.com,IOS,Apple,iPhone XS,INACTIVE,2024-12-15T16:20:00Z


## 5. Device Platform Distribution

Overview of mobile device platforms in your environment.

In [5]:
LookoutEvents
| where EventType == "DEVICE"
| summarize arg_max(TimeGenerated, *) by DeviceGuid
| summarize Count = count() by DevicePlatform, DeviceManufacturer
| sort by Count desc
| render piechart

Pie Chart rendered - Device distribution showing IOS (42%), Samsung Android (35%), Google Android (13%), OnePlus (6%), Xiaomi (4%)